In [2]:
# Read all the text book
words = open('constitution.txt', 'r').read().split()
len(words)

132548

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
len(words)

In [ ]:
words[33]

In [ ]:
# Build a voculabory of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
len(chars)

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['~']= 0
itos = {i:s for s,i in stoi.items()}
itos
# assert '~' in stoi.items().key()
# chars

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device='cpu'
device

# device = torch.cuda.current_device()
# device

In [ ]:
#build the dataset
block_size = 100  # context length
X,Y =[],[]
for w in words:
    # print(w)
    context = [0] * block_size
    for ch in w + '~':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), "---->", itos[ix])
        context = context[1:] + [ix]
# stoi['~']
# itos
X = torch.tensor(X, device=device)
Y = torch.tensor(Y, device=device)


In [ ]:
X.shape, X.dtype, Y.shape , Y.dtype, Y.device

In [ ]:
X

In [ ]:
Y

In [ ]:
X.shape, Y.shape

In [ ]:
## we have 87 characters. Lets embed them in 2 dimensions first.
C = torch.randn((87,4), device=device)
C.shape

In [ ]:
V = F.one_hot(torch.tensor(5, device=device), num_classes=87).float() @ C
V
# we are discarding the one as the results are same

In [ ]:
emb = C[X]
emb.shape

In [ ]:
## use view to changed the shape for matmull correct dimensions
g = torch.cuda.manual_seed(42)
W1 = torch.randn((20, 400), generator=g, device=device)
b1 = torch.randn((400), device=device, generator=g)
W2 = torch.randn((400,87), generator=g, device=device)
b2 = torch.randn((87), generator=g, device=device)

parameters = [C,W1,b1,W2,b2]
for p in parameters:
    p.requires_grad=True
# h = torch.tanh(emb.view(-1,40) @ W1 +b1)
# h.shape

In [ ]:
# No of parameter
sum(p.nelement() for p in parameters)

In [ ]:
lre = torch.linspace(-3,0, 100)
lrs = 10**lre

In [ ]:
lri , lossi, stepi = [], [], []

In [ ]:
import torch.nn.functional as F
for i in range(50):
    
    #minibatch construct
    # ix = torch.randint(0, X.shape[0],(64,))
        
    #forward pass
    emb = C[X]
    # print(f'emb.shape: {emb.shape}')
    # print(f'{emb.device} | {}')
    ##        multiple of NO of batch * dimension      = 20
    h = torch.tanh(emb.view(-1,20) @ W1 + b1)
    # print(f'h : {h}')
    logits = h @ W2 +b2
    # print(f'logits.shape : {logits.shape}')
    # print(Y.shape)
    loss = F.cross_entropy(logits,Y)

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    # lr = lrs[i]
    #Emperiacally Observed .01 seems ro be a better Learning rate
    for p in parameters:
        p.data += -0.05 * p.grad
    
    #track status
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.item())
        
    # print(f'lr: {lr} | loss : {loss.item()}')
    if i % 5 == 0:
        print(f'step : {i} |loss: {loss.item()}')

In [ ]:
# step_cpu = stepi.detach.numpy()
# lossi_cpu=lossi.cpu()
# plt.plot(stepi_cpu,lossi_cpu)
len(stepi), len(lossi)
# stepi.append(100)
lossi.append(5.556219100952148)
lossi.append(5.556219100952148)
plt.plot(stepi, lossi)

In [ ]:

C_cpu = C.cpu()
plt.figure(figsize=(8,8))
plt.scatter(C_cpu[:,0].data, C_cpu[:,1].data, s=200)
print(C_cpu.shape[0])
for i in range(C_cpu.shape[0]):
    print(C_cpu[i,0].item())
    plt.text(C_cpu[i,0].item(). C_cpu[i,1].item(), itos[i], ha="center", va='center', color='white')
plt.grid('minor')

In [ ]:
context = [0] * block_size
C[torch.tensor([context])].shape

In [ ]:
torch.Size([1, 3, 10])


In [ ]:
# sample from the model
g = torch.cuda.manual_seed(42)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))